<a href="https://colab.research.google.com/github/gouravdhalwal08/Cricket-Match-Outcome-Predictor-using-Exact-Inference/blob/main/Cricket_Match_Outcome_Predictor_using_Exact_Inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install pgmpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 18.0 MB/s eta 0:00:00


In [3]:
# Import necessary libraries
from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination

In [4]:
# Step 1: Define the structure of the Bayesian Network
# Nodes: Team Strength, Weather, Pitch Condition, Outcome
model = BayesianNetwork([('Team Strength', 'Outcome'),
                         ('Weather', 'Outcome'),
                         ('Pitch Condition', 'Outcome')])

In [5]:

# Step 2: Define the Conditional Probability Distributions (CPDs)

# CPD for Team Strength (1: Strong, 0: Weak)
cpd_team_strength = TabularCPD(variable='Team Strength', variable_card=2,
                               values=[[0.7], [0.3]])  # 70% Strong, 30% Weak

In [6]:
print(cpd_team_strength)

+------------------+-----+
| Team Strength(0) | 0.7 |
+------------------+-----+
| Team Strength(1) | 0.3 |
+------------------+-----+


In [7]:
# CPD for Weather (1: Good, 0: Bad)
cpd_weather = TabularCPD(variable='Weather', variable_card=2,
                         values=[[0.8], [0.2]])  # 80% Good, 20% Bad

In [8]:
print(cpd_weather)

+------------+-----+
| Weather(0) | 0.8 |
+------------+-----+
| Weather(1) | 0.2 |
+------------+-----+


In [9]:
# CPD for Pitch Condition (1: Favorable, 0: Unfavorable)
cpd_pitch_condition = TabularCPD(variable='Pitch Condition', variable_card=2,
                                 values=[[0.6], [0.4]])  # 60% Favorable, 40% Unfavorable


In [10]:
print(cpd_pitch_condition)

+--------------------+-----+
| Pitch Condition(0) | 0.6 |
+--------------------+-----+
| Pitch Condition(1) | 0.4 |
+--------------------+-----+


In [11]:
# CPD for Outcome (1: Win, 0: Loss)
cpd_outcome = TabularCPD(variable='Outcome', variable_card=2,
                         values=[[0.9, 0.7, 0.6, 0.4, 0.3, 0.1, 0.5, 0.2],  # Prob of Win
                                 [0.1, 0.3, 0.4, 0.6, 0.7, 0.9, 0.5, 0.8]],  # Prob of Loss
                         evidence=['Team Strength', 'Weather', 'Pitch Condition'],
                         evidence_card=[2, 2, 2])

In [12]:
print(cpd_outcome)

+-----------------+--------------------+-----+--------------------+--------------------+
| Team Strength   | Team Strength(0)   | ... | Team Strength(1)   | Team Strength(1)   |
+-----------------+--------------------+-----+--------------------+--------------------+
| Weather         | Weather(0)         | ... | Weather(1)         | Weather(1)         |
+-----------------+--------------------+-----+--------------------+--------------------+
| Pitch Condition | Pitch Condition(0) | ... | Pitch Condition(0) | Pitch Condition(1) |
+-----------------+--------------------+-----+--------------------+--------------------+
| Outcome(0)      | 0.9                | ... | 0.5                | 0.2                |
+-----------------+--------------------+-----+--------------------+--------------------+
| Outcome(1)      | 0.1                | ... | 0.5                | 0.8                |
+-----------------+--------------------+-----+--------------------+--------------------+


In [13]:
# Step 3: Add CPDs to the model
model.add_cpds(cpd_team_strength, cpd_weather, cpd_pitch_condition, cpd_outcome)


In [14]:

# Step 4: Validate the model
assert model.check_model()

In [15]:
# Step 5: Perform Exact Inference using Variable Elimination
inference = VariableElimination(model)

In [16]:
# Example Queries:
# Query 1: Given that the team is strong, weather is good, and the pitch is favorable,
# What is the probability of winning?
result = inference.query(variables=['Outcome'], evidence={'Team Strength': 1, 'Weather': 1, 'Pitch Condition': 1})
print("\nPrediction with Strong Team, Good Weather, Favorable Pitch:")
print(result)


Prediction with Strong Team, Good Weather, Favorable Pitch:
+------------+----------------+
| Outcome    |   phi(Outcome) |
+============+================+
| Outcome(0) |         0.2000 |
+------------+----------------+
| Outcome(1) |         0.8000 |
+------------+----------------+


In [17]:
# Query 2: Given that the team is weak, weather is bad, and the pitch is unfavorable,
# what is the probability of winning?
result = inference.query(variables=['Outcome'], evidence={'Team Strength': 0, 'Weather': 0, 'Pitch Condition': 0})
print("\nPrediction with Weak Team, Bad Weather, Unfavorable Pitch:")
print(result)



Prediction with Weak Team, Bad Weather, Unfavorable Pitch:
+------------+----------------+
| Outcome    |   phi(Outcome) |
+============+================+
| Outcome(0) |         0.9000 |
+------------+----------------+
| Outcome(1) |         0.1000 |
+------------+----------------+


In [18]:

# Query 3: Given that the team is strong and the weather is bad (ignoring pitch condition),
# what is the probability of winning?
result = inference.query(variables=['Outcome'], evidence={'Team Strength': 1, 'Weather': 0})
print("\nPrediction with Strong Team, Bad Weather (Pitch Unknown):")
print(result)


Prediction with Strong Team, Bad Weather (Pitch Unknown):
+------------+----------------+
| Outcome    |   phi(Outcome) |
+============+================+
| Outcome(0) |         0.2200 |
+------------+----------------+
| Outcome(1) |         0.7800 |
+------------+----------------+


In [23]:
# Step 6: Function to compute accuracy based on actual outcomes
def compute_accuracy(model, test_data):
    inference = VariableElimination(model)
    correct_predictions = 0
    total_predictions = len(test_data)

    for data in test_data:
        # Extract evidence and actual outcome
        evidence = {'Team Strength': data['Team Strength'],
                    'Weather': data['Weather'],
                    'Pitch Condition': data['Pitch Condition']}
        actual_outcome = data['Outcome']

        # Perform inference to get the predicted outcome
        result = inference.query(variables=['Outcome'], evidence=evidence)
        predicted_outcome = result.values.argmax()  # 0 for Loss, 1 for Win

        # Check if prediction matches the actual outcome
        if predicted_outcome == actual_outcome:
            correct_predictions += 1

    # Compute accuracy as a percentage
    accuracy = (correct_predictions / total_predictions) * 100
    return accuracy

# Updated, more diverse test data (Team Strength, Weather, Pitch Condition, Actual Outcome)
test_data = [
    {'Team Strength': 1, 'Weather': 1, 'Pitch Condition': 1, 'Outcome': 1},  # Strong team, good weather, favorable pitch, Win
    {'Team Strength': 0, 'Weather': 0, 'Pitch Condition': 0, 'Outcome': 0},  # Weak team, bad weather, unfavorable pitch, Loss
    {'Team Strength': 1, 'Weather': 0, 'Pitch Condition': 1, 'Outcome': 0},  # Strong team, bad weather, favorable pitch, Loss (unexpected)
    {'Team Strength': 0, 'Weather': 1, 'Pitch Condition': 1, 'Outcome': 1},  # Weak team, good weather, favorable pitch, Win (unexpected)
    {'Team Strength': 0, 'Weather': 0, 'Pitch Condition': 1, 'Outcome': 0},  # Weak team, bad weather, favorable pitch, Loss
]

# Calculate accuracy on updated test data
accuracy = compute_accuracy(model, test_data)
print(f"\nModel Accuracy: {accuracy:.2f}%")



Model Accuracy: 80.00%
